__качаем объявления о продаже квартир с авито.ru__

_Evgeny S. Borisov <parser@mechanoid.su>_

In [1]:
# !pacman -S firefox firefox-i18n-r  geckodriver
# !pip install seleniuam

In [2]:
# import re
import sys
import logging
from datetime import datetime as dtm
from tqdm.notebook import tqdm
import pandas as pd

In [3]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', '{:.2f}'.format)
tqdm.pandas()

logging.basicConfig(
        format=u'[%(levelname)-8s] %(asctime)s | %(message)s',
        datefmt='%Y-%m-%d %H:%M:%S',
        level=logging.INFO,
        # level=logging.DEBUG,
        stream=sys.stdout,
    )

---

In [4]:
# url = (
# 'https://www.avito.ru/yaroslavskaya_oblast/avtomobili/chevrolet/niva'
# '?bt=0'
# '&i=1'
# '&pmax=150000'
# '&pmin=10000'
# )

In [5]:
profile_path = '/home/mechanoid/.mozilla/firefox/p144xo2m.default-release'

base_url = 'https://www.avito.ru/'
place = 'sevastopol'
avito_path = place+'/kvartiry/prodam'

raw_data_path = 'data/raw/flat/avito' # каталог с файлами объявлений

In [9]:
%%time 

from lib.downloader import DownloaderSeleniumFirefox
from lib.avito import AvitoParserRealty

df,html = AvitoParserRealty(
        base_url=base_url+'/'+avito_path,
        driver=DownloaderSeleniumFirefox(profile_path)
    ).load(req_param='',keep_html=True)# , npage_end=2,)

print(len(df))

[INFO    ] 2023-03-29 16:00:45 | DownloaderSeleniumFirefox: downloader init
[INFO    ] 2023-03-29 16:00:45 | DownloaderSeleniumFirefox: open virtual browser
[INFO    ] 2023-03-29 16:00:47 | AdsListParser: downloader init
[INFO    ] 2023-03-29 16:00:47 | AvitoParser: init
[INFO    ] 2023-03-29 16:00:51 | AvitoParser: 87 pages for read
[INFO    ] 2023-03-29 16:00:51 | AdsListParser: start read and parse pages...
[INFO    ] 2023-03-29 16:00:53 | AdsListParser: read page 1
[INFO    ] 2023-03-29 16:00:56 | AdsListParser: read page 2
[INFO    ] 2023-03-29 16:00:59 | AdsListParser: read page 3
[INFO    ] 2023-03-29 16:01:03 | AdsListParser: read page 4
[INFO    ] 2023-03-29 16:01:05 | AdsListParser: read page 5
[INFO    ] 2023-03-29 16:01:08 | AdsListParser: read page 6
[INFO    ] 2023-03-29 16:01:15 | AdsListParser: read page 7
[INFO    ] 2023-03-29 16:01:18 | AdsListParser: read page 8
[INFO    ] 2023-03-29 16:01:22 | AdsListParser: read page 9
[INFO    ] 2023-03-29 16:01:25 | AdsListParser

In [11]:
# df

In [12]:
print(len(df))
df = df.dropna().drop_duplicates(['avito_id','description']).reset_index(drop=True)
print(len(df))

3896
3180


In [13]:
df['place']=place

In [14]:
ts = dtm.now().strftime('%Y-%m-%d_%H-%M')
assert len(df)>0
df.to_excel(f'{raw_data_path}/avito_{ts}_raw.xlsx',index=False)

In [15]:
df.sample(2)

,avito_id,title,price,obj_name,adr,description,page,ts,place
611,2715671323,"Объявление «1-к. квартира, 40,3 м², 6/9 эт.» 6 фотографий",5642000,ЖК «Porto Franco»,"ул. Братьев Манганари, д. 28","Квартира у моря в Севастополе! От застройщика! \n\nЖК «Porto Franco» — первый в Севастополе жилой проект у моря с собственной набережной и инфраструктурой целого города. \n\nУлучшенный проект ЖК «Porto Franco»: \n⁃ монолитно-каркасные дома;\n⁃ потолки — 2,7 м;\n⁃",13,2023-03-29 16:04:54.059167,sevastopol
439,2924749139,"Объявление «Апартаменты-студия, 34 м², 4/10 эт.» 10 фотографий",6500000,,"ул. Адмирала Фадеева, 48","Продаётся студия в Парк-отель. \nВозле дома есть вся необходимая инфракструктура: остановка, супермаркеты, магазины, рынок, аптека, кофейня, кафе, Парк Победы, аквапарк, пляж, сады и школы. Всю необходимую информацию предоставим по телефону. Звоните! Торг в",9,2023-03-29 16:04:54.059167,sevastopol


In [16]:
# !pip install dash_table